In [35]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(
  api_key=os.environ['HF_API_KEY'],  # this is also the default, it can be omitted
  base_url = "https://router.huggingface.co/v1"
  )

deployment="openai/gpt-oss-20b:fireworks-ai"

In [ ]:
# define tools 
function_schema = { 
    "type":"function",
    "name": "get_word_of_the_day",
    "description" : "Retrieves the word of the day based in a given language",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "language" : {
                "type": "string",
                "enum" : ["french", "english", "italian"],
                "description" : "The language"
                } 
            },
            "required": ["language"],
            "additionalProperties" : False
        },
    "strict": True
}


# Harmony-style system message
system_prompt = f"""
You are a function-calling assistant.
You have the following function available:

Name: {function_schema['name']}
Description: {function_schema['description']}
Parameters: {json.dumps(function_schema['parameters'], indent=2)}

When a user asks something that matches this function, respond ONLY in valid JSON with:
{{
  "name": "<function_name>",
  "arguments": {{...}}
}}
Do not add extra text or explanation.
"""

def get_word_of_the_day(language: str) -> str:
    return f"In {language}: the word of the day is 'GOOD'"



# 1st prompt with tools

prompt = "What is the word of the day in English ?"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt}
] 

response =  client.chat.completions.create(   
    model=deployment,                                         
    messages=messages,
    max_tokens=200
)



In [ ]:
response_message = json.loads(response.choices[0].message.content)
if response_message['name']:
    print(f"Recommended Function Call : {response_message['name']}")

    available_functions = {
        'get_word_of_the_day': get_word_of_the_day
    }

    function_to_call = available_functions[response_message['name']]
    function_args = response_message['arguments']

    function_response = function_to_call(**function_args)
    print(function_response)


Recommended Function Call : get_word_of_the_day
In english: the word of the day is 'GOOD'
